# NFL Big Data Bowl 2024 - Tackling

Introduction

## Cleaning

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
tracking = []
for i in range(1, 10):
    tracking.append(pd.read_csv(f"data/tracking_week_{i}.csv"))
tracking = pd.concat(tracking)
tracking.reset_index(drop=True, inplace=True)

In [3]:
from Cleaning import clean_games_data, clean_plays_data, clean_players_data, \
    clean_tackles_data, clean_tracking_data
games = clean_games_data(pd.read_csv("data/games.csv"))
players = clean_players_data(pd.read_csv("data/players.csv"))
plays = clean_plays_data(pd.read_csv("data/plays.csv"))
tackles = clean_tackles_data(pd.read_csv("data/tackles.csv"))
tracking = clean_tracking_data(tracking)

Games data has been cleaned and memory has been reduced by 2.65625 bytes.
Players data has been cleaned and memory has been reduced by 13.1484375 bytes.
Plays data has been cleaned and memory has been reduced by 1041.80859375 bytes.
Tackles data has been cleaned and memory has been reduced by 476.4921875 bytes.
Tracking data has been cleaned and memory has been reduced by 476070.234375 bytes.


In [ ]:
from Cleaning import check_for_snap, check_for_end
full_plays = check_for_snap(plays, tracking)
full_plays = check_for_end(full_plays, tracking)

## Feature Engineering

In [4]:
# Make all the plays go from left to right to remove the variablilty it may cause
from Preprocessing import all_plays_left_to_right
from Preprocessing import create_acceleration_vectors, create_velocity_vectors, create_player_influence

full_plays, full_tracking = all_plays_left_to_right(plays, tracking)

full_tracking = create_acceleration_vectors(full_tracking)
full_tracking = create_velocity_vectors(full_tracking)
full_tracking = create_player_influence(full_tracking)

In [ ]:
from Visualizations import animate_play

animate_play(games, full_plays, full_tracking, gameId=2022090800, playId=56, acceleration=True, velocity=True)

In [ ]:
data = pd.merge(plays, tracking, on = ["gameId", "playId"])

Finding who made the tackle on this play

Show the movement of the players during the play

In [ ]:
# Make a more customized version in plotly

In [ ]:
exampleFrame = data.query('gameId == 2022091103 and '
           'playId == 3126 and '
           'frameId == 20')

In [ ]:
from Visualizations import animate_play
animate_play(games, full_plays, full_tracking, gameId=2022091103, playId=3126, velocity=True)

In [ ]:
play = data[(data['gameId'] == 2022090800) & (data['playId'] == 80)]

In [ ]:
playerData = data[(data['gameId'] == 2022090800) & (data['playId'] == 56) & (data['nflId'] == 35472)]

Find the relationship between the player and the ball at all times.

In [ ]:
football = play[play['club'] == 'football']

footballAndPlayer = pd.merge(football, play, on = ['gameId', 'playId', 'ballCarrierId', 'ballCarrierDisplayName',
       'playDescription', 'quarter', 'down', 'yardsToGo', 'possessionTeam',
       'defensiveTeam', 'yardlineSide', 'yardlineNumber', 'gameClock',
       'preSnapHomeScore', 'preSnapVisitorScore', 'passResult', 'passLength',
       'penaltyYards', 'prePenaltyPlayResult', 'playResult',
       'playNullifiedByPenalty', 'absoluteYardlineNumber', 'offenseFormation',
       'defendersInTheBox', 'passProbability', 'preSnapHomeTeamWinProbability',
       'preSnapVisitorTeamWinProbability', 'homeTeamWinProbabilityAdded',
       'visitorTeamWinProbilityAdded', 'expectedPoints', 'expectedPointsAdded',
       'foulName1', 'foulName2', 'foulNFLId1', 'foulNFLId2',
        'frameId', 'time',
       'playDirection','event'], suffixes=('_football', '_player'))

In [ ]:
footballAndPlayer

In [ ]:
footballAndPlayer['distanceFromBall'] = np.sqrt((footballAndPlayer['x_x'] - footballAndPlayer['x_y'])**2 + (footballAndPlayer['y_x'] - footballAndPlayer['y_y'])**2)

Find the distance between the players and the ball at all times

In [ ]:
defense = footballAndPlayer[footballAndPlayer['club_y'] == footballAndPlayer.iloc[0]['defensiveTeam']]

Plot the distance between each player and the ball at all times during the play

In [ ]:
playersInGame = set(defense['displayName_y'])
for index, player in enumerate(playersInGame):
    plt.plot(defense[defense['displayName_y'] == player]['frameId'], defense[defense['displayName_y'] == player]['distanceFromBall'])
plt.legend(playersInGame)
plt.axhline(0)
plt.axvline(defense[defense['event'] == "tackle"]['frameId'].iloc[0])
plt.show()

Plot the speed of the player at all times

In [ ]:
playersInGame = set(defense['displayName_y'])
for index, player in enumerate(playersInGame):
    plt.plot(defense[defense['displayName_y'] == player]['frameId'], defense[defense['displayName_y'] == player]['s_y'])
plt.legend(playersInGame)
plt.axvline(defense[defense['event'] == "tackle"]['frameId'].iloc[0])
plt.show()

Plot the acceleration of the player at all times

In [ ]:
playersInGame = set(defense['displayName_y'])
for index, player in enumerate(playersInGame):
    plt.plot(defense[defense['displayName_y'] == player]['frameId'], defense[defense['displayName_y'] == player]['a_y'])
plt.legend(playersInGame)
plt.axvline(defense[defense['event'] == "tackle"]['frameId'].iloc[0])
plt.show()

# Classify each tackle as open-field, sideline, in the hole, solo, or gang.

Criteria for each tackle:
- open field
    - The defender is 
    
ToDo:
Use a semi-supervised algorothm to classify all tackles
Use this classification to rank players on their ability to make these specific tackles
Create a reinforcemnt learning agent to make each type of tackle and see what the computer can do to make each tackle